In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

##Baseline

In [2]:
# Training settings

batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(320 , 10) # 20 * 4 * 4  # http://taewan.kim/post/cnn/

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

In [4]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [5]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # nll_loss?? / cross entropy loss와의 관계 확인!
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299930
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.313562
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.300389
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.295995
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.291314
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.289033
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.269902
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.265857
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.251798
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.239546
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.200689
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.175704
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.149290
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.037836
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.951609
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.736515
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.538280
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.415755
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.013400
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.923265
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2098, Accuracy: 9317/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.235646
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.159915
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.222784
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.174633
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.248922
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.193274
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.124557
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.199159
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.149716
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.206454
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.166572
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.151342
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.175005
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.243358
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.135943
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.204307
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.198621
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.434533
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.082162
Train Epoch: 2 [12

##TASK 1

In [8]:
class revised_Net(nn.Module):
  def __init__(self):
    super(revised_Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
    self.mp1 = nn.MaxPool2d(kernel_size =2, stride = 2)
    self.mp2 = nn.MaxPool2d(kernel_size =2, stride = 2, padding =1)
    self.fc1 = nn.Linear(128*4*4, 100)
    self.fc2 = nn.Linear(100, 10)
  

  def forward(self, x):
    in_size = x.size(0)
    x =F.relu(self.mp1(self.conv1(x)))
    x = F.relu(self.mp1(self.conv2(x)))
    x = F.relu(self.mp2(self.conv3(x)))
    x = x.view(in_size ,-1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    return F.log_softmax(x)

In [ ]:
model = revised_Net()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
      
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # nll_loss?? / cross entropy loss와의 관계 확인!
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300570
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295923
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.315682
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.308279
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.307269
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.302120
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.290240
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.312692
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.296627
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.287215
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.298307
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.281672
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.292122
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.281343
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.278911
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.280848
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.285131
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.286492
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.273207
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.269079
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.2414, Accuracy: 9274/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.414985
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.306185
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.119526
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.283777
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.189601
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.156599
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.261506
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.169755
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.211936
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.255653
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.158244
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.116598
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.161786
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.198739
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.229055
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.150452
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.144531
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.251240
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.142747
Train Epoch: 2 [12

# **Task 2** :
베이스라인 마음대로 변경 + Softmax regression 과제에서 사용했던 데이터셋 (변경 가능하나 같은 데이터셋으로 진행하시는 걸 추천드립니다!) >> CNN으로 성공해보자!

지난번 과제와 마찬가지로 다양한 방식을 시도해보시고 그 시도 기록을 남겨주세요. 가장 성능이 좋았던 모델도 표시 부탁드립니다!

이번엔 GPU resource 문제가 생길 수 있습니다. 파라미터 조정, dropout 추가 등 연산 효율성도 고려해서 베이스라인 조정해보세요!

In [9]:
!pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 82 kB 735 kB/s 
     |████████████████████████████████| 11.2 MB 10.5 MB/s 
     |████████████████████████████████| 61 kB 10.1 MB/s 
     |████████████████████████████████| 15.7 MB 24.5 MB/s 
     |████████████████████████████████| 9.9 MB 25.9 MB/s 
     |████████████████████████████████| 944 kB 54.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
   

In [10]:
import d2l

In [11]:
#라이브러리
%matplotlib inline
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display()

In [12]:
# Training settings

batch_size = 18

trans = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(
    root = "../data",
    train = True,
    transform = trans,
    download = True)

testset = torchvision.datasets.MNIST(
    root = "../data",
    train = False,
    transform = trans,
    download = True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [13]:
print(train_dataset[0][0].shape, test_dataset[0][0].shape)
print(len(train_dataset), len(test_dataset))

torch.Size([1, 28, 28]) torch.Size([1, 28, 28])
60000 10000


##Model

In [14]:
class Net1(nn.Module):
  def __init__(self):
    super(Net1, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding=1)
    self.mp = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(4*4*64 , 10)

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

In [15]:
model = Net1()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [16]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

In [17]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data 
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:
for epoch in range(1, 10):
    train(epoch)
    test()

AttributeError: ignored